# Code for using FiftyOne to train a Faster RCNN on COCO data

###  Imports

In [1]:
import torch
import fiftyone as fo
import fiftyone.zoo as foz
import torchvision.models.detection.roi_heads
from fiftyone import ViewField as F

from dataset import FiftyOneTorchDataset, get_transforms
from model import create_model
from utils import add_detections

from engine import train_model
import config

torch.manual_seed(1)

### Load full dataset from model zoo

In [2]:
#Lodad in the dataset from the FiftyOne model Zoo
fo_dataset = foz.load_zoo_dataset("coco-2017", "validation")

#needed to calculate image height and width
fo_dataset.compute_metadata()

session = fo.launch_app(fo_dataset)

Found annotations at 'C:\Users\blain\fiftyone\coco-2017\raw\instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [19.9s elapsed, 0s remaining, 303.1 samples/s]      
Dataset 'coco-2017-validation' created


For example, cluttered images make it difficult for models to localize objects. We can use FiftyOne to create a view containing only samples with more than, say, 10 objects. You can perform the same operations on views as datasets, so we can create an instance of our PyTorch dataset from this view:

In [3]:
#if we want to see images with more than 10 items, we can
# busy_view = fo_dataset.match(F("ground_truth.detections").length() > 10)
# busy_torch_dataset = FiftyOneTorchDataset(busy_view)
# session.view = busy_view

### Create training and testing views (and corresponding PyTorch datasets) that only contain some items from the full dataset

In [4]:
subset = False

train_transforms, test_transforms = get_transforms()

if subset:
    # to filter certain items from the dataset we can
    item_list = ["car", "dog", "bus", 'fork', 'tie', 'person']
    item_list = ['airplane']
    item_view = fo_dataset.filter_labels("ground_truth",
            F("label").is_in(item_list))

    #session.view = item_view

    # split the dataset in train and test set
    train_view = item_view.take((len(item_view) * config.TRAIN_TEST_SPLIT), seed=51)
    test_view = item_view.exclude([s.id for s in train_view])

else:
    train_view = fo_dataset.take(len(fo_dataset) * config.TRAIN_TEST_SPLIT)
    test_view = fo_dataset.exclude([s.id for s in train_view])
    item_list = fo_dataset.distinct("ground_truth.detections.label")

print(f'Traning on {len(train_view)} samples')
print(f'Testing on {len(test_view)} samples')

# use our dataset and defined transformations
train_dataset = FiftyOneTorchDataset(train_view, train_transforms,
        classes=item_list)
evaluation_dataset = FiftyOneTorchDataset(test_view, test_transforms,
        classes=item_list)

#this is needed for later use, but not for creating the dataset
if item_list[0] != 'background':
     item_list.insert(0,'background')

Traning on 4000 samples
Testing on 1000 samples


In [5]:
# # map labels to single vehicle class
# vehicle_list = ['car', 'bus', 'truck']
# vehicles_map = {c: "vehicle" for c in vehicle_list}
#
# train_map_view = train_view.map_labels("ground_truth", vehicles_map)
# test_map_view = test_view.map_labels("ground_truth", vehicles_map)
#
# # use our dataset and defined transformations
# torch_map_dataset = FiftyOneTorchDataset(train_map_view, train_transforms)
# torch_map_dataset_test = FiftyOneTorchDataset(test_map_view, test_transforms)

### Training and Evaluation

In [6]:
# #to change the loss function, create a new function and implement like below
# import torchvision
# torchvision.models.detection.roi_heads.fastrcnn_loss = cliprcnn_loss

In [ ]:
MODEL_TYPE='CLIP-FRCNN'
# CLIP-Backbone-FRCNN creates a FRCNN using CLIP features as the model backbone
# CLIP-FRCNN creates a FRCNN using CLIP features as the model backbone, and embeds the rois using CLIP's embedding
# Fully custom vanilla uses a pre-trained resnet50 backbone, and generates new anchor generator and roi pooling
# Custom-Vanilla uses the pre-trained FRCNN from pytorch and replaces the roi heads only
#
import clip
text_tokens = clip.tokenize(["This is " + desc for desc in item_list]).cuda()

model = create_model(MODEL_TYPE, text_tokens)
test = False


if test:
    train_model(model, evaluation_dataset, evaluation_dataset, num_epochs=config.NUM_EPOCHS, MODEL_TYPE=MODEL_TYPE, batch_size=4)
else:
    train_model(model, train_dataset, evaluation_dataset, num_epochs=config.NUM_EPOCHS, MODEL_TYPE=MODEL_TYPE, batch_size=4)

Using device cuda


C:\Users\blain\anaconda3\envs\torch-frcnn\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Training Epoch: [0]  [   0/1000]  eta: 0:22:47  lr: 0.000010  loss: 0.8858 (0.8858)  loss_classifier: 0.0000 (0.0000)  loss_box_reg: 0.0617 (0.0617)  loss_objectness: 0.7377 (0.7377)  loss_rpn_box_reg: 0.0864 (0.0864)  time: 1.3673  data: 0.2691  max mem: 4717
Training Epoch: [0]  [  10/1000]  eta: 0:08:16  lr: 0.000060  loss: 0.8588 (0.7875)  loss_classifier: 0.0000 (0.0000)  loss_box_reg: 0.1218 (0.1250)  loss_objectness: 0.5458 (0.5511)  loss_rpn_box_reg: 0.0864 (0.1114)  time: 0.5015  data: 0.1417  max mem: 5190
Training Epoch: [0]  [  20/1000]  eta: 0:07:14  lr: 0.000110  loss: 0.6416 (0.6753)  loss_classifier: 0.0000 (0.0000)  loss_box_reg: 0.1185 (0.1212)  loss_objectness: 0.3375 (0.4085)  loss_rpn_box_reg: 0.1021 (0.1456)  time: 0.3968  data: 0.1279  max mem: 5190
Training Epoch: [0]  [  30/1000]  eta: 0:06:52  lr: 0.000160  loss: 0.6087 (0.6955)  loss_classifier: 0.0000 (0.0000)  loss_box_reg: 0.1305 (0.1449)  loss_objectness: 0.2259 (0.3619)  loss_rpn_box_reg: 0.1898 (0.1887)

In [ ]:
MODEL_TYPE='CLIP-Backbone-FRCNN'

model = create_model(MODEL_TYPE, classes=item_list)
test = False

if test:
    train_model(model, evaluation_dataset, evaluation_dataset, num_epochs=config.NUM_EPOCHS, MODEL_TYPE=MODEL_TYPE, batch_size=2)
else:
    train_model(model, train_dataset, evaluation_dataset, num_epochs=config.NUM_EPOCHS, MODEL_TYPE=MODEL_TYPE, batch_size=2)

In [ ]:
#train a custom vanilla model so that we can compare and make sure the CLIP FRCNN is comparable
# Fully-Custom-Vanilla is most appropriate as it generates the model in a similar fashion
MODEL_TYPE = 'Fully-Custom-Vanilla'

vanilla_model = create_model(MODEL_TYPE, classes=item_list)
train_model(vanilla_model, train_dataset, evaluation_dataset, num_epochs=10, MODEL_TYPE=MODEL_TYPE)

### Evaluate the model

In [ ]:
add_detections(model, evaluation_dataset, fo_dataset, field_name="predictions")

results = fo.evaluate_detections(
    test_view,
    "predictions",
    classes=item_list,
    eval_key="eval",
    compute_mAP=True
)

In [ ]:
results.mAP()

In [ ]:
results.print_report()

By default, objects are only matched with other objects of the same class. In order to get an interesting confusion matrix, we need to match interclass objects by setting `classwise=False`.

In [ ]:
results_interclass = fo.evaluate_detections(
    test_view, 
    "predictions", 
    classes=item_list,
    compute_mAP=True, 
    classwise=False
)

In [ ]:
plot = results.plot_pr_curves(classes=item_list)
plot.show()

In [ ]:
results_interclass.plot_confusion_matrix(classes=item_list, include_other=False, include_missing=False)

The [detection evaluation](https://voxel51.com/docs/fiftyone/user_guide/evaluation.html#detections) also added the attributes `eval_fp`, `eval_tp`, and `eval_fn` to every predicted detection indicating if it is a false positive, true positive, or false negative. 
Let's create a view to find the worst samples by sorting by `eval_fp` using the [FiftyOne App](https://voxel51.com/docs/fiftyone/user_guide/app.html) to visualize the results. 

In [ ]:
session.view = test_view.sort_by("eval_fp", reverse=True)

In [ ]:
session.view = test_view.sort_by("eval_fp", reverse=True)

It would be best to get this [data reannotated to fix these mistakes](https://towardsdatascience.com/managing-annotation-mistakes-with-fiftyone-and-labelbox-fc6e87b51102), but in the meantime, we can easily remedy this by simply creating a new view that remaps the labels `car`, `truck`, and `bus` all to `vehicle` and then retraining the model with that. This is only possible because we are backing our data in FiftyOne and loading views into PyTorch as needed. Without FiftyOne, the PyTorch dataset class or the underlying data would need to be changed to remap these classes.

In [ ]:
# map labels to single vehicle class
vehicle_list = ['car', 'bus', 'truck']
vehicles_map = {c: "vehicle" for c in vehicle_list}

train_map_view = train_view.map_labels("ground_truth", vehicles_map)
test_map_view = test_view.map_labels("ground_truth", vehicles_map)

# use our dataset and defined transformations
torch_map_dataset = FiftyOneTorchDataset(train_map_view, train_transforms)
torch_map_dataset_test = FiftyOneTorchDataset(test_map_view, test_transforms)

In [ ]:
# Only 2 classes (background and vehicle)
MODEL_TYPE = 'Vanilla-FRCNN'
vehicle_model = create_model(MODEL_TYPE, num_classes=(len(vehicles_map)+1))
train_model(vehicle_model, torch_map_dataset, torch_map_dataset_test, num_epochs=2, MODEL_TYPE=MODEL_TYPE)

In [ ]:
add_detections(vehicle_model, torch_map_dataset_test, test_map_view, field_name="vehicle_predictions")

In [ ]:
vehicle_results = fo.evaluate_detections(
    test_map_view, 
    "vehicle_predictions", 
    classes=["vehicle"], 
    eval_key="vehicle_eval", 
    compute_mAP=True
)

In [ ]:
vehicle_results.mAP()

In [ ]:
vehicle_results.print_report()

Due to our ability to easily visualize and manage our dataset with FiftyOne, we were able to spot and take action on a dataset issue that would otherwise have gone unnoticed if we only concerned ourselves with dataset-wide evaluation metrics and fixed dataset representations. Through these efforts, we managed to increase the mAP of the model to 43%.

Even though this example workflow may not work in all situations, this kind of class-merging strategy can be effective in cases where more fine-grained discrimination is not called for.